<p style="font-weight: bold; background-color: yellow; padding: 10px; display: inline;">TODO</p>

- mention the whole application e2e with a nice clean diagram
- CTA to Anyscale

<p style="font-weight: bold; background-color: yellow; padding: 10px; display: inline;">TODO</p>

- explain how Workspaces work
- mention compute and env (and new containerfile)
- mention no need to worry about infra (unlike k8s stack -- mention weak points about k8s stack)
- mention weak points about sagemaker, vertex and DB (without naming dropping DB for now)
- show how to configure Git

In [15]:
%load_ext autoreload
%autoreload all

<p style="font-weight: bold; background-color: yellow; padding: 10px; display: inline;">TODO</p>

- explain what autoreload does

In [1]:
import os
import ray
import sys
import warnings
from utils import show
sys.path.append(os.path.abspath("../"))
warnings.filterwarnings("ignore")

<div class="alert alert-block alert-warning"> <b> Title</b> 

Description [link descirption](url). Example in `italics`.

</div>

<p style="font-weight: bold; background-color: yellow; padding: 10px; display: inline;">TODO</p>

- explain ray run_time env
- can RAY_DEBUG always be turned on?
- remove train v2 flag once it becomes available (and the .env file making code)
- add a small section on env vars (dotenv) somewhere
- link to distributed debugger

```python
from dotenv import load_dotenv
load_dotenv()

ray.init(
    runtime_env={
        "env_vars": {
            "RAY_DEBUG": "1",
            "RAY_TRAIN_V2_ENABLED": "1",
        },
    },
)
```

In [3]:
# Shutdown this notebook
import IPython
IPython.get_ipython().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

: 